In [14]:
import bs4
import urllib3
from datetime import date
import re
import csv

In [3]:
# code by KI to get stock codes or import a csv file
import pandas as pd 
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)
code_df = code_df[['회사명', '종목코드']]
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})

In [8]:
def get_max_page(code):
    
    target = "http://finance.naver.com/item/board.nhn?code=" + code + "&page=1"
    pm = urllib3.PoolManager()
    html = pm.urlopen(url=target, method="GET").data
    soup = bs4.BeautifulSoup(html, 'lxml')
    a_tag = soup.find_all("a")
    for a in a_tag:
        if a.has_attr("href") and a.text.strip("\n\t") == "맨뒤":
            target_tag = a
    text = target_tag["href"]
    match=re.search('[\d]+$',text)
    max_page = match.group()
    
    return max_page

In [11]:
def scrape_discussion(code):
    page_num = 1
    post_list = []
    while page_num < int(get_max_page(code)): ###
        target = "http://finance.naver.com/item/board.nhn?code=" + code + "&page=" + str(page_num)
        pm = urllib3.PoolManager()
        html = pm.urlopen(url=target, method="GET").data
        soup = bs4.BeautifulSoup(html, 'lxml')
        div_tag = soup.find_all("div", class_ ="section inner_sub")
        div_row = div_tag[0].find_all("tr")
        post_list = []
        for row in div_row[2:25]:
            if div_row.index(row) not in [7, 13, 19]:
                post_dic = {}
                info_list = row.find_all('td')
                time = info_list[0].text
                date_list = time.split(" ")[0].split(".")
                pre_date = date(int(date_list[0]), int(date_list[1]), int(date_list[2]))
                today = date.today()
                if (today - pre_date).days >= 10: ###set a day
                    post_dic["time"] = time
                    post_dic["title"] = info_list[1].find_all("a")[0]["title"]
                    post_dic["recommend"] = info_list[2].text
                    post_dic["user_id"] = info_list[3].text.strip("\n\t")
                    post_dic["click"] = info_list[4].text
                    post_dic["like"] = info_list[5].text
                    post_dic["dislike"] = info_list[6].text
                    post_list.append(post_dic)
                else:
                    return post_list
        page_num += 1
   
    return post_list  

In [23]:
def get_all_stock_forum():
    
    fieldnames = ["time", "title", "recommend", "user_id", "click", "like", "dislike"]
    for stock in code_df.code:
        post_list = scrape_discussion(stock)
        file_name = code_df[code_df.code == stock].name.values[0] + ".csv"
        with open(file_name, "w") as f:
            writer = csv.DictWriter(f, fieldnames = fieldnames)
            writer.writeheader()
            for post in post_list:
                row = {}
                row["time"] = post.get("time")
                row["title"] = post.get("title")
                row["recommend"] = post.get("recommend")
                row["user_id"] = post.get("user_id")
                row["click"] = post.get("click")
                row["like"] = post.get("like")
                row["dislike"] = post.get("dislike")
                writer.writerow(row)
    return

In [24]:
get_all_stock_forum()

KeyboardInterrupt: 